In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping


In [3]:
# DATA load
X = pd.read_csv('./content/loan_train_preprocessed.csv')

# backward
X = X[['term', 'initial_list_status', 'int_rate', 
'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 
'inq_last_6mths', 'revol_util', 'recoveries', 
'collection_recovery_fee', 'tot_cur_bal', 
'home_ownershipRENT', 'purposesmall_business', 
'purposewedding', 'earliest_cr_line2000']]

y = pd.read_csv('./content/loan_train_label.csv')
y = y.drop(['id'], axis=1)

In [4]:
# Dividing the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20 )
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.20 )

x_train.shape,y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape

((10240, 16), (10240, 1), (2560, 16), (2560, 1), (3200, 16), (3200, 1))

In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:

import warnings
warnings.filterwarnings("ignore")



model = Sequential()
model.add(Dense(2048, input_shape = (len(x_train.columns),), activation='ELU'))
model.add(Dense(1024, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='ELU'))
model.add(Dense(1, activation = 'sigmoid'))


model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy', f1])

In [7]:
# Adjust the weights of the classes since your dataset is HIGHLY IMBALANCED!
class_weight = {0: 1.,
                1: 1.}

model.fit(x_train.values,
         y_train.values,
         epochs = 300,
         batch_size = 2048,
         validation_data = (x_val.values, y_val.values), class_weight=class_weight,
         callbacks=[EarlyStopping(monitor='val_f1', mode='max', patience=50, restore_best_weights=True)])


Epoch 1/300
5/5 [==============================] - 2s 210ms/step - loss: 0.6728 - accuracy: 0.6011 - f1: 0.5937 - val_loss: 0.5996 - val_accuracy: 0.6750 - val_f1: 0.6344
Epoch 2/300
5/5 [==============================] - 1s 161ms/step - loss: 0.6192 - accuracy: 0.6416 - f1: 0.6180 - val_loss: 0.5915 - val_accuracy: 0.6762 - val_f1: 0.6155
Epoch 3/300
5/5 [==============================] - 1s 170ms/step - loss: 0.5987 - accuracy: 0.6607 - f1: 0.6298 - val_loss: 0.5646 - val_accuracy: 0.6742 - val_f1: 0.6432
Epoch 4/300
5/5 [==============================] - 1s 169ms/step - loss: 0.5844 - accuracy: 0.6604 - f1: 0.6334 - val_loss: 0.5652 - val_accuracy: 0.6750 - val_f1: 0.6764
Epoch 5/300
5/5 [==============================] - 1s 167ms/step - loss: 0.5761 - accuracy: 0.6704 - f1: 0.6503 - val_loss: 0.5578 - val_accuracy: 0.6762 - val_f1: 0.6909
Epoch 6/300
5/5 [==============================] - 1s 173ms/step - loss: 0.5726 - accuracy: 0.6669 - f1: 0.6528 - val_loss: 0.5536 - val_accuracy

In [8]:
y_prediction = model.predict(x_test.values)
y_prediction= [1 if i>=0.5 else 0 for i in y_prediction]
print("The Test Accuracy of the model is: {} %".format(accuracy_score(y_test.values, y_prediction) * 100.)) 
print()

print(confusion_matrix(y_test.values, y_prediction))
print()

target_names = ['Fully Paid', 'Charged Off']
print(classification_report(y_test, y_prediction, target_names=target_names))

The Test Accuracy of the model is: 65.53125 %

[[ 825  741]
 [ 362 1272]]

              precision    recall  f1-score   support

  Fully Paid       0.70      0.53      0.60      1566
 Charged Off       0.63      0.78      0.70      1634

    accuracy                           0.66      3200
   macro avg       0.66      0.65      0.65      3200
weighted avg       0.66      0.66      0.65      3200



## test dataset prediction

In [9]:
# DATA load
target_X = pd.read_csv('./content/loan_test_preprocessed.csv')

# backward
target_X = target_X[['term', 'initial_list_status', 'int_rate', 
'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 
'inq_last_6mths', 'revol_util', 'recoveries', 
'collection_recovery_fee', 'tot_cur_bal', 
'home_ownershipRENT', 'purposesmall_business', 
'purposewedding', 'earliest_cr_line2000']]

target_y = pd.read_csv('./content/loan_test_label.csv')
target_y = target_y.drop(['id'], axis=1)

In [10]:
y_prediction = model.predict(target_X.values)
y_prediction= [1 if i>=0.5 else 0 for i in y_prediction]

